In [ ]:
import pandas as pd
import os
import io
import re
import pandas as pd
import soundfile as sf
from IPython.display import Audio

file_path = "/data/hf_cache/yodas-granary/data/en000/asr_only/00000000.parquet"
df = pd.read_parquet(file_path)
# wav_paths = df["audio"].apply(lambda x: x["path"])
print(df.columns)
print(df.head())
print(os.path.exists(file_path))
print(os.path.getsize(file_path) / (1024**2), "MB")  # 查看文件大小


Index(['utt_id', 'audio', 'duration', 'lang', 'task', 'text', 'translation_en',
       'original_audio_id', 'original_audio_offset'],
      dtype='object')
                                    utt_id  \
0   en000_00000000_Y0aGwNq86f4_112_62_5_54   
1  en000_00000000_Y0aGwNq86f4_118_16_30_48   
2  en000_00000000_Y0aGwNq86f4_158_96_39_48   
3    en000_00000000_Y0aGwNq86f4_16_00_6_00   
4  en000_00000000_Y0aGwNq86f4_199_02_34_98   

                                               audio  duration  lang   task  \
0  {'bytes': b'RIFF\xa4\xb4\x02\x00WAVEfmt \x10\x...      5.54  <en>  <asr>   
1  {'bytes': b'RIFF$\xe2\x0e\x00WAVEfmt \x10\x00\...     30.48  <en>  <asr>   
2  {'bytes': b'RIFF$G\x13\x00WAVEfmt \x10\x00\x00...     39.48  <en>  <asr>   
3  {'bytes': b'RIFF$\xee\x02\x00WAVEfmt \x10\x00\...      6.00  <en>  <asr>   
4  {'bytes': b'RIFF\xa4\x14\x11\x00WAVEfmt \x10\x...     34.98  <en>  <asr>   

                                                text translation_en  \
0  Which sites have y

In [ ]:
file_name = df["audio"].iloc[0]["path"]
# print(f"file_name : {file_name}")
print(df.dtypes)
print(df.iloc[41])

file_name : en000_00000000_Y0aGwNq86f4_112_62_5_54.wav


In [3]:
wav_paths = os.path.join(
    file_path,
    file_name
)
print(wav_paths)

/data/hf_cache/yodas-granary/data/en000/asr_only/00000000.parquet/en000_00000000_Y0aGwNq86f4_112_62_5_54.wav


In [4]:
# 取第一个样本
sample = df.iloc[0]
print(sample["audio"].keys())
print(sample["text"])
audio_bytes = sample["audio"]["bytes"]

# 用 BytesIO 包装成“文件对象”
audio_file = io.BytesIO(audio_bytes)

# 用 soundfile 读取
audio, sr = sf.read(audio_file)
print("Audio shape:", audio.shape)

# 如果在 Jupyter 中播放
Audio(audio, rate=sr)

dict_keys(['bytes', 'path'])
Which sites have you been using and how did the use of these technologies assist your search?
Audio shape: (88640,)


In [5]:
# ====== 1. CONFIG ======
PARQUET_PATH = file_path
OUTPUT_DIR = "/data/user_data/haolingp/outputs/mfa_corpus_test"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ====== 2. Load parquet ======
df = pd.read_parquet(PARQUET_PATH)

# 只取前 10 条
df = df.iloc[:20]

# ====== 3. Text cleaning ======
def clean_text(t):
    t = t.lower()
    t = re.sub(r"[^a-z' ]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

# ====== 4. For each sample, write wav + lab ======
for i, row in df.iterrows():

    # ---- audio bytes ----
    audio_bytes = row["audio"]["bytes"]
    audio_file = io.BytesIO(audio_bytes)

    # decode wav
    audio, sr = sf.read(audio_file)

    # ---- text ----
    text = clean_text(row["text"])
    # text = row["text"]

    # ---- file names ----
    uttid = f"utt_{i}"
    wav_path = os.path.join(OUTPUT_DIR, f"{uttid}.wav")
    lab_path = os.path.join(OUTPUT_DIR, f"{uttid}.lab")

    # ---- write wav ----
    sf.write(wav_path, audio, sr)


    # ---- write lab ----
    with open(lab_path, "w") as f:
        f.write(text)

    print(f"Wrote {wav_path} and {lab_path}")

print("Done! Corpus created at:", OUTPUT_DIR)

Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_0.wav and /data/user_data/haolingp/outputs/mfa_corpus_test/utt_0.lab
Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_1.wav and /data/user_data/haolingp/outputs/mfa_corpus_test/utt_1.lab
Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_2.wav and /data/user_data/haolingp/outputs/mfa_corpus_test/utt_2.lab
Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_3.wav and /data/user_data/haolingp/outputs/mfa_corpus_test/utt_3.lab
Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_4.wav and /data/user_data/haolingp/outputs/mfa_corpus_test/utt_4.lab
Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_5.wav and /data/user_data/haolingp/outputs/mfa_corpus_test/utt_5.lab
Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_6.wav and /data/user_data/haolingp/outputs/mfa_corpus_test/utt_6.lab
Wrote /data/user_data/haolingp/outputs/mfa_corpus_test/utt_7.wav and /data/user_data/haolingp/outputs/mf

## use MFA
conda install -c conda-forge montreal-forced-aligner
mfa version

mfa model download acoustic english_us_arpa
mfa model download dictionary english_us_arpa

# 检查文件是否存在：
ls ~/.local/share/mfa/models/acoustic
ls ~/.local/share/mfa/models/dictionary

🚀 Step 3 — 准备你的 corpus 目录

我们之前生成的是：

mfa_corpus_test/
    utt_0.wav
    utt_0.lab
    utt_1.wav
    utt_1.lab
    ...


📌 每一个 .wav 必须对应一个 .lab。

终端执行：

先删除：rm -rf /data/user_data/haolingp/outputs/mfa_output
rm -rf ~/.local/share/mfa



### “mfa align <corpus_dir> <dictionary> <acoustic_model> <output_dir>”


##‘’mfa align mfa_corpus_test english_us_arpa english_us_arpa mfa_output‘’


mfa align \
    /data/user_data/haolingp/outputs/mfa_corpus_test \
    english_us_arpa \
    english_us_arpa \
    /data/user_data/haolingp/outputs/mfa_output
    --clean

要加clean， 不然的话他不会重新跑，只会用以前跑过的数据

In [10]:
import tgt

tg = tgt.read_textgrid("/data/user_data/haolingp/outputs/mfa_output/utt_000008.TextGrid")

words = tg.get_tier_by_name("words")
phones = tg.get_tier_by_name("phones")


#     print(interval.text, interval.start_time, interval.end_time)

# filter out the "" empty words
words_clean = [w for w in words.intervals if w.text.strip() != ""]

for interval in words_clean:
    print(interval)
    


Interval(0.08, 0.23, "the")
Interval(0.23, 0.4, "other")
Interval(0.4, 0.64, "thing")
Interval(0.64, 0.91, "about")
Interval(0.91, 1.47, "linkedin")
Interval(1.47, 1.71, "even")
Interval(1.71, 1.84, "as")
Interval(1.84, 2.0, "a")
Interval(2.0, 2.48, "student")
Interval(2.52, 2.8, "is")
Interval(2.8, 2.98, "who")
Interval(2.98, 3.09, "your")
Interval(3.09, 3.68, "connections")
Interval(3.68, 3.94, "are")


In [27]:
def load_word_alignment(textgrid_path):
    tg = tgt.read_textgrid(textgrid_path)
    words = tg.get_tier_by_name("words").intervals
    out = []
    for w in words:
        if w.text.strip():
            out.append({
                "word": w.text,
                "start": w.start_time,
                "end": w.end_time,
            })
    return out

words_dict = load_word_alignment("/data/user_data/haolingp/outputs/mfa_output/utt_0.TextGrid")

load_word_alignment("/data/user_data/haolingp/outputs/mfa_output/utt_0.TextGrid")



[{'word': 'which', 'start': 0.0, 'end': 0.11},
 {'word': 'sites', 'start': 0.11, 'end': 0.45},
 {'word': 'have', 'start': 0.45, 'end': 0.68},
 {'word': 'you', 'start': 0.68, 'end': 0.79},
 {'word': 'been', 'start': 0.79, 'end': 1.0},
 {'word': 'using', 'start': 1.0, 'end': 1.49},
 {'word': 'and', 'start': 1.73, 'end': 1.87},
 {'word': 'how', 'start': 1.87, 'end': 2.08},
 {'word': 'did', 'start': 2.08, 'end': 2.35},
 {'word': 'the', 'start': 2.35, 'end': 2.44},
 {'word': 'use', 'start': 2.44, 'end': 2.7},
 {'word': 'of', 'start': 2.7, 'end': 2.87},
 {'word': 'these', 'start': 2.87, 'end': 3.05},
 {'word': 'technologies', 'start': 3.05, 'end': 3.69},
 {'word': 'assist', 'start': 3.76, 'end': 4.16},
 {'word': 'your', 'start': 4.16, 'end': 4.27},
 {'word': 'search', 'start': 4.27, 'end': 4.78}]

构造1 s 的 segmentation

In [21]:
def build_1s_segments(words):
    max_time = words[-1]["end"]
    t = 0
    segments = []

    while t < max_time:
        seg_words = []
        for w in words:
            # 值落在 [t, t+1) 区间内
            if w["start"] < t + 1 and w["end"] > t:
                seg_words.append(w["word"])
        segments.append({"second": t, "words": seg_words})
        t += 1
    
    return segments

build_1s_segments(words_dict)


[{'second': 0, 'words': ['which', 'sites', 'have', 'you', 'been']},
 {'second': 1, 'words': ['using', 'and', 'how']},
 {'second': 2, 'words': ['how', 'did', 'the', 'use', 'of', 'these']},
 {'second': 3, 'words': ['these', 'technologies', 'assist']},
 {'second': 4, 'words': ['assist', 'your', 'search']}]

## 接下来需要得到所有的 llm chunk的 start time 和end time

In [8]:
import re

def normalize_text(s):
    """lowercase & keep only letters"""
    s = s.lower()
    s = re.sub(r"[^a-z ]+", " ", s)
    return s.strip()

def match_llm_chunks_to_mfa(llm_chunks, mfa_words):
    """
    llm_chunks: list of strings (e.g., ["House and riches", "are an inheritance", ...])
    mfa_words: list of dicts:
      [{"word": "house", "start":0.0, "end":0.2}, ...]
    """
    # 1. Normalize MFA words
    mfa_tokens = [normalize_text(w["word"]) for w in mfa_words]

    results = []

    for chunk in llm_chunks:
        # ---- step 1: tokenize LLM chunk ----
        tokens = normalize_text(chunk).split()

        # ---- step 2: find matching MFA words ----
        matched_indices = []
        for t in tokens:
            for i, w in enumerate(mfa_tokens):
                if w == t:
                    matched_indices.append(i)
                    break  # only use first match, avoid duplicates

        # ---- step 3: if no match found ----
        if not matched_indices:
            results.append({
                "chunk": chunk,
                "start": None,
                "end": None
            })
            continue

        # ---- step 4: compute chunk time ----
        start = min(mfa_words[i]["start"] for i in matched_indices)
        end   = max(mfa_words[i]["end"]   for i in matched_indices)

        results.append({
            "chunk": chunk,
            "start": start,
            "end": end
        })

    return results

In [ ]:
# to test the accuracy
llm_chunks = [
    "which",
    "sites",
    "have",
    "you",
    "been",
    "using",
    "and",
    "how",
    "did",
    "the",
    "use",
    "of",
    "these",
    "technologies",
    "assist",
    "your",
    "search"
  ]

aligned_chunks_one = match_llm_chunks_to_mfa(llm_chunks, words_dict)

for item in aligned_chunks_one:
    print(item)

{'chunk': 'which', 'start': 0.0, 'end': 0.11}
{'chunk': 'sites', 'start': 0.11, 'end': 0.45}
{'chunk': 'have', 'start': 0.45, 'end': 0.68}
{'chunk': 'you', 'start': 0.68, 'end': 0.79}
{'chunk': 'been', 'start': 0.79, 'end': 1.0}
{'chunk': 'using', 'start': 1.0, 'end': 1.49}
{'chunk': 'and', 'start': 1.73, 'end': 1.87}
{'chunk': 'how', 'start': 1.87, 'end': 2.08}
{'chunk': 'did', 'start': 2.08, 'end': 2.35}
{'chunk': 'the', 'start': 2.35, 'end': 2.44}
{'chunk': 'use', 'start': 2.44, 'end': 2.7}
{'chunk': 'of', 'start': 2.7, 'end': 2.87}
{'chunk': 'these', 'start': 2.87, 'end': 3.05}
{'chunk': 'technologies', 'start': 3.05, 'end': 3.69}
{'chunk': 'assist', 'start': 3.76, 'end': 4.16}
{'chunk': 'your', 'start': 4.16, 'end': 4.27}
{'chunk': 'search', 'start': 4.27, 'end': 4.78}


In [23]:
import json

json_path = "/data/user_data/haolingp/outputs/llm_segmentation_json/utt_000000.json"

with open(json_path, "r", encoding="utf-8") as f:
    segmentation_json = json.load(f)

llm_chunks = segmentation_json["low_latency"]["English"]
aligned_chunks_low = match_llm_chunks_to_mfa(llm_chunks, words_dict)

for item in aligned_chunks_low:
    print(item)

{'chunk': 'Which sites', 'start': 0.0, 'end': 0.45}
{'chunk': 'have you been using', 'start': 0.45, 'end': 1.49}
{'chunk': 'and', 'start': 1.73, 'end': 1.87}
{'chunk': 'how', 'start': 1.87, 'end': 2.08}
{'chunk': 'did the use', 'start': 2.08, 'end': 2.7}
{'chunk': 'of these technologies', 'start': 2.7, 'end': 3.69}
{'chunk': 'assist', 'start': 3.76, 'end': 4.16}
{'chunk': 'your search?', 'start': 4.16, 'end': 4.78}


## 按 1 秒切割 emit 结果，构造 streaming trajectory， 采用保守策略

In [24]:
def assign_chunks_by_second(aligned_chunks):
    """
    aligned_chunks: 输出列表，每个包含:
      {"chunk": str, "start": float, "end": float}

    返回:
      [{"second": int, "emit": [chunk1, chunk2, ...]}, ...]
    """
    # 找到最大时间（决定多少秒）
    max_time = max(x["end"] for x in aligned_chunks)
    max_second = int(max_time) + 1

    emitted = []        # 已经 emit 的 chunk index
    timeline = []       # 输出

    for sec in range(max_second):
        sec_end = sec + 1.0

        to_emit = []
        for i, item in enumerate(aligned_chunks):
            if i in emitted:
                continue

            # 满足 mentor 的保守策略:
            # chunk 完整结束 <= 当前秒
            if item["end"] <= sec_end:
                to_emit.append(item["chunk"])
                emitted.append(i)

        timeline.append({"second": sec, "emit": to_emit})

    return timeline


timeline = assign_chunks_by_second(aligned_chunks_low)
# print(timeline)
assign_chunks_by_second(aligned_chunks_low)

[{'second': 0, 'emit': ['Which sites']},
 {'second': 1, 'emit': ['have you been using', 'and']},
 {'second': 2, 'emit': ['how', 'did the use']},
 {'second': 3, 'emit': ['of these technologies']},
 {'second': 4, 'emit': ['assist', 'your search?']}]

In [ ]:
##检查一下
{'chunk': 'which', 'start': 0.0, 'end': 0.11}
{'chunk': 'sites', 'start': 0.11, 'end': 0.45}
{'chunk': 'have', 'start': 0.45, 'end': 0.68}
{'chunk': 'you', 'start': 0.68, 'end': 0.79}
{'chunk': 'been', 'start': 0.79, 'end': 1.0}
{'chunk': 'using', 'start': 1.0, 'end': 1.49}
{'chunk': 'and', 'start': 1.73, 'end': 1.87}
{'chunk': 'how', 'start': 1.87, 'end': 2.08}
{'chunk': 'did', 'start': 2.08, 'end': 2.35}
{'chunk': 'the', 'start': 2.35, 'end': 2.44}
{'chunk': 'use', 'start': 2.44, 'end': 2.7}
{'chunk': 'of', 'start': 2.7, 'end': 2.87}
{'chunk': 'these', 'start': 2.87, 'end': 3.05}
{'chunk': 'technologies', 'start': 3.05, 'end': 3.69}
{'chunk': 'assist', 'start': 3.76, 'end': 4.16}
{'chunk': 'your', 'start': 4.16, 'end': 4.27}
{'chunk': 'search', 'start': 4.27, 'end': 4.78}

{'chunk': 'search', 'start': 4.27, 'end': 4.78}

In [26]:
def build_final_segments(timeline, segmentation_json):
    """
    timeline: [{'second': int, 'emit': [chunk1, chunk2, ...]}]
    segmentation_json: LLM segmentation (low_latency)

    return: source_segments, target_segments
    """

    eng_chunks = segmentation_json["low_latency"]["English"]
    zh_chunks  = segmentation_json["low_latency"]["Chinese"]

    # 构建中英 map
    eng2zh = {e: z for e, z in zip(eng_chunks, zh_chunks)}

    source_segments = []
    target_segments = []

    for entry in timeline:
        eng_list = entry["emit"]
        
        # 1) 英文拼接
        eng_concat = " ".join(eng_list).strip()

        # 2) 中文拼接
        zh_concat_list = []
        for e in eng_list:
            zh_concat_list.append(eng2zh.get(e, ""))

        zh_concat = "".join(zh_concat_list).strip()

        source_segments.append(eng_concat)
        target_segments.append(zh_concat)
    
    out = {
        "source": source_segments,
        "target": target_segments
    }

    return out



import json

json_path = "/data/user_data/haolingp/outputs/llm_segmentation_json/utt_000000.json"

with open(json_path, "r", encoding="utf-8") as f:
    segmentation_json = json.load(f)

print(build_final_segments(timeline, segmentation_json))

{'source': ['Which sites', 'have you been using and', 'how did the use', 'of these technologies', 'assist your search?'], 'target': ['哪些网站', '你一直在使用以及', '如何这些技术的使用', '协助了', '你的搜索？']}
